In [3]:
import pandas as pnds
import openpyxl
import datetime
import pickle
import duplicated_history

# 날짜 셋팅
work_date = datetime.datetime.now() - datetime.timedelta(2)
target_date = work_date.strftime('%Y%m%d')                              # 어제 날짜 포맷
dfdata_dir = f'./data/{target_date}/dfdata/'                            # 읽어들일 dfdata디렉토리 './data/YYYYMMDD/dfdata/'

# 5. KICC로부터 생성한 신용카드 승인내역 dataframe
card_xl_filename = 'df_kicc_history_' + target_date                     # 파일 이름: df_kicc_history_YYYYMMDD

#   KICC 카드 승인내역 Dataframe loading
try:
    with open(dfdata_dir + card_xl_filename, "rb") as file:
        card_history_df = pickle.load(file)
except Exception as e:
    with open('./error.log', 'a') as file:
        file.write(
            f'[edi_opera.py - Reading Data] <{datetime.datetime.now()}> pickle file-reading error ({card_xl_filename}) ===> {e}\n'
        )
    raise(e)
print(card_history_df)

#   이전(2일 전) 거래내역 중, 이미 등록된 어제 날짜 내역 제거
del_dup_serials = duplicated_history.get_dup_serials(work_date)
if len(del_dup_serials) != 0:                                           # 이전 내역에 포함된 내역이 있을 경우
    card_history_df = card_history_df[card_history_df['거래고유번호'].isin(del_dup_serials) == False]           # isin()결과가 False 인 것만 추출
print('dup derial : ', del_dup_serials)

#   내일 날짜는 제외 - 해당 날짜의 거래내역만 추출
tomorrow = (work_date + datetime.timedelta(1)).strftime('%d')                                                   # 내일 날짜의 day: 'DD' (두자리 문자열)
card_history_df = card_history_df[card_history_df['거래고유번호'].str.startswith(tomorrow, na=False) == False]  # 거래고유번호가 내일날자(DD)로 시작되지 않는것들만
print(card_history_df)

          거래고유번호 승인구분                   date                     카드번호   발급카드사  \
98  230021596446   승인  2022-07-23   00:02:08  5303-6929-1576-3285      신한카드법인   
97  230033394392   승인  2022-07-23   00:33:18  5557-9776-8294-0601       해외마스타   
96  230033400626   승인  2022-07-23   00:44:15  5557-9774-0415-3897       해외마스타   
95  230512835294   승인  2022-07-23   05:00:09  5395-9374-1530-9249       해외마스타   
94  230521863681   승인  2022-07-23   05:48:37  5557-9775-4321-0335       해외마스타   
..           ...  ...                    ...                      ...     ...   
4   241921446712   승인  2022-07-24   19:10:43  5567-1708-9695-1949       해외마스타   
3   241933531087   승인  2022-07-24   19:13:34  5405-4250-0973-4301       해외마스타   
2   242111293472   승인  2022-07-24   21:45:54  4579-7207-0862-9064      KB국민카드   
1   242221471129   승인  2022-07-24   22:43:20  3791-8399-5202-220        삼성아멕스   
0   242221475687   승인  2022-07-24   22:45:21  3791-8399-5202-220        삼성아멕스   

    매입카드사      금액 할부개월     